<a href="https://colab.research.google.com/github/dn3ndra/DeepLearning/blob/main/03.%20Week%203/Assignment/CIFAR_10_Datasets_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ADC

In [1]:
# Install library jika diperlukan (khusus Google Colab)
!pip install torch torchvision scikit-learn matplotlib

# Import library
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Gunakan transformasi ini saat ambil data
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)


trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Label nama kelas
classes = trainset.classes


100%|██████████| 170M/170M [00:13<00:00, 13.0MB/s]


🧠 Model 1: CNN (Convolutional Neural Network)

In [3]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x


🧠 Model 2: MLP (Multilayer Perceptron) Vanilla

In [4]:
class MLPVanilla(nn.Module):
    def __init__(self):
        super(MLPVanilla, self).__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 3, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        return self.fc(x)


⚙️ Training & Evaluation Function

In [5]:
def train(model, optimizer, criterion, loader, device):
    model.train()
    running_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

def evaluate(model, loader, device):
    model.eval()
    y_true, y_pred, y_score = [], [], []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_score.extend(probs.cpu().numpy())

    return np.array(y_true), np.array(y_pred), np.array(y_score)


🚀 Jalankan Training untuk CNN

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training Loop
for epoch in range(10):
    loss = train(model, optimizer, criterion, trainloader, device)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")


Epoch 1, Loss: 1.5062
Epoch 2, Loss: 1.0414
Epoch 3, Loss: 0.8603
Epoch 4, Loss: 0.7463
Epoch 5, Loss: 0.6692
Epoch 6, Loss: 0.6103
Epoch 7, Loss: 0.5693
Epoch 8, Loss: 0.5281
Epoch 9, Loss: 0.4927
Epoch 10, Loss: 0.4553


✅ Evaluasi CNN

In [7]:
y_true, y_pred, y_score = evaluate(model, testloader, device)

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, average='macro')
rec = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

# ROC AUC Macro (multi-class)
try:
    auc = roc_auc_score(y_true, y_score, multi_class='ovo', average='macro')
except:
    auc = 'Error (label mismatch)'

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC Score: {auc}")


Accuracy: 0.8256
Precision: 0.8306
Recall: 0.8256
F1 Score: 0.8232
AUC Score: 0.9833068833333335


✅ CNN Test Accuracy

In [8]:
# Tambahkan akurasi dalam bentuk persen
acc_percent = acc * 100
print(f"CNN Test Accuracy : {acc_percent:.2f}%")

CNN Test Accuracy : 82.56%


🚀 Training untuk MLP Vanilla

In [9]:
model_mlp = MLPVanilla().to(device)
optimizer_mlp = optim.Adam(model_mlp.parameters(), lr=0.001)

for epoch in range(10):
    loss = train(model_mlp, optimizer_mlp, criterion, trainloader, device)
    print(f"MLP Epoch {epoch+1}, Loss: {loss:.4f}")


MLP Epoch 1, Loss: 1.8238
MLP Epoch 2, Loss: 1.6499
MLP Epoch 3, Loss: 1.5708
MLP Epoch 4, Loss: 1.5274
MLP Epoch 5, Loss: 1.4931
MLP Epoch 6, Loss: 1.4638
MLP Epoch 7, Loss: 1.4421
MLP Epoch 8, Loss: 1.4246
MLP Epoch 9, Loss: 1.4096
MLP Epoch 10, Loss: 1.3941


✅ Evaluasi MLP

In [10]:
# Evaluasi MLP
y_true_mlp, y_pred_mlp, y_score_mlp = evaluate(model_mlp, testloader, device)

acc_mlp = accuracy_score(y_true_mlp, y_pred_mlp)
prec_mlp = precision_score(y_true_mlp, y_pred_mlp, average='macro')
rec_mlp = recall_score(y_true_mlp, y_pred_mlp, average='macro')
f1_mlp = f1_score(y_true_mlp, y_pred_mlp, average='macro')

# ROC AUC untuk MLP
try:
    auc_mlp = roc_auc_score(y_true_mlp, y_score_mlp, multi_class='ovo', average='macro')
except:
    auc_mlp = 'Error (label mismatch)'

# Output hasil evaluasi
print(f"MLP Accuracy: {acc_mlp:.4f}")
print(f"MLP Precision: {prec_mlp:.4f}")
print(f"MLP Recall: {rec_mlp:.4f}")
print(f"MLP F1 Score: {f1_mlp:.4f}")
print(f"MLP AUC Score: {auc_mlp}")


MLP Accuracy: 0.4429
MLP Precision: 0.4642
MLP Recall: 0.4429
MLP F1 Score: 0.4261
MLP AUC Score: 0.8604015777777779


In [11]:
# Konversi akurasi ke persentase
acc_percent_mlp = acc_mlp * 100
print(f"MLP Accuracy (in %): {acc_percent_mlp:.2f}%")

MLP Accuracy (in %): 44.29%


# Kesimpulan

Berdasarkan hasil evaluasi yang diperoleh dari model **CNN** dan **MLP** pada dataset **CIFAR-10**, berikut adalah beberapa poin utama yang dapat disimpulkan:

## 1. **Model CNN:**
- **Akurasi**: Model CNN mencapai akurasi **82.56%**, yang menunjukkan bahwa model ini cukup baik dalam mengenali pola dan fitur pada dataset CIFAR-10.
- **Precision**: Dengan **83.06%**, model CNN memiliki precision yang baik, artinya sebagian besar prediksi positif yang dilakukan oleh model benar-benar positif.
- **Recall**: **82.56%**, yang menunjukkan bahwa model mampu mendeteksi sebagian besar data positif yang ada.
- **F1 Score**: **82.32%**, ini menunjukkan keseimbangan yang baik antara precision dan recall. Model CNN mampu menjaga kinerja baik pada kedua metrik tersebut.
- **AUC Score**: **98.33%**, yang sangat tinggi dan menunjukkan bahwa model CNN memiliki kemampuan sangat baik dalam membedakan antara kelas-kelas yang ada, dengan sedikit kesalahan dalam memprediksi kelas negatif.

## 2. **Model MLP:**
- **Akurasi**: Model MLP hanya mencapai **44.29%** akurasi, yang jauh lebih rendah dibandingkan dengan CNN. Hal ini menunjukkan bahwa MLP kurang efektif untuk tugas klasifikasi gambar seperti CIFAR-10.
- **Precision**: Dengan **46.42%**, MLP menunjukkan kemampuan yang terbatas dalam memberikan prediksi positif yang benar.
- **Recall**: **44.29%**, ini menunjukkan bahwa MLP kurang mampu mendeteksi sebagian besar data positif yang ada.
- **F1 Score**: **42.61%**, yang juga mencerminkan bahwa model ini tidak seimbang dalam hal precision dan recall.
- **AUC Score**: **86.04%**, meskipun lebih rendah dari CNN, AUC untuk MLP masih menunjukkan bahwa model ini cukup baik dalam membedakan kelas-kelas, meskipun tidak sebaik CNN.

## 3. **Kesimpulan Umum:**
- **CNN** menunjukkan kinerja yang jauh lebih baik dibandingkan dengan **MLP** pada dataset **CIFAR-10**, dengan akurasi yang jauh lebih tinggi dan metrik evaluasi lainnya yang mendekati nilai maksimum.
- **MLP**, meskipun lebih sederhana, kesulitan untuk mencapai hasil yang baik pada tugas klasifikasi gambar yang kompleks. Ini menunjukkan bahwa **MLP tidak cocok untuk menangani data gambar**, karena tidak memiliki mekanisme untuk menangkap fitur spasial dalam gambar seperti yang dilakukan oleh **CNN**.
- Oleh karena itu, untuk tugas klasifikasi gambar yang kompleks seperti CIFAR-10, **CNN adalah pilihan yang lebih baik** karena dapat memanfaatkan fitur spasial dan hubungan antar piksel dalam gambar.

Secara keseluruhan, **CNN** terbukti lebih unggul dalam mengatasi masalah klasifikasi gambar pada dataset CIFAR-10.


# Penjelasan Persamaan Matematika
## 1. **Cross-Entropy Loss (Loss Function)**
Cross-Entropy Loss digunakan untuk mengukur perbedaan antara distribusi probabilitas prediksi model dan distribusi probabilitas label yang sebenarnya.

$$
\text{Loss} = - \sum_{i=1}^{N} y_i \cdot \log(p_i)
$$

- \( y_i \): Nilai sebenarnya (label) untuk kelas ke-\(i\) (biasanya dalam bentuk one-hot encoding).
- \( p_i \): Probabilitas prediksi untuk kelas ke-\(i\) (hasil dari fungsi Softmax).
- \( N \): Jumlah kelas.
- **Fungsi ini mengukur seberapa baik model dalam memprediksi kelas yang benar.** Semakin kecil nilai loss, semakin baik performa model.

## 2. **Accuracy**
Akurasi mengukur seberapa banyak prediksi yang benar dibandingkan dengan total data yang ada.

$$
\text{Accuracy} = \frac{\text{Jumlah Prediksi Benar}}{\text{Jumlah Data Total}}
$$

- **Akurasi** menunjukkan **seberapa sering model memberikan prediksi yang benar**.

## 3. **Precision**
Precision mengukur seberapa banyak prediksi yang benar untuk kelas positif dibandingkan dengan jumlah semua prediksi positif.

$$
\text{Precision} = \frac{TP}{TP + FP}
$$

- \( TP \) (True Positives): Jumlah kasus di mana model dengan benar memprediksi kelas positif.
- \( FP \) (False Positives): Jumlah kasus di mana model salah memprediksi kelas positif (seharusnya negatif).
- Precision menunjukkan **seberapa banyak prediksi positif yang benar** dari total prediksi positif yang dibuat oleh model.

## 4. **Recall**
Recall mengukur seberapa banyak data positif yang benar-benar terdeteksi oleh model dari semua data yang seharusnya positif.

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

- \( FN \) (False Negatives): Jumlah kasus di mana model salah memprediksi kelas negatif padahal data tersebut seharusnya positif.
- Recall menunjukkan **seberapa baik model dalam mendeteksi kelas positif**.

## 5. **F1 Score**
F1 Score adalah rata-rata harmonis dari Precision dan Recall. F1 score memberikan gambaran yang lebih lengkap mengenai keseimbangan antara Precision dan Recall.

$$
\text{F1 Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$

- F1 score memberikan **ukuran tunggal** yang menggambarkan keseimbangan antara Precision dan Recall. Nilai F1 akan mendekati 1 jika kedua metrik tersebut tinggi.

## 6. **AUC (Area Under Curve) dan ROC (Receiver Operating Characteristic)**
AUC adalah **area di bawah kurva ROC**. Kurva ROC menunjukkan hubungan antara **True Positive Rate (TPR)** dan **False Positive Rate (FPR)**, yang digunakan untuk mengukur kinerja model dalam membedakan antara kelas positif dan negatif.

$$
\text{TPR (Recall)} = \frac{TP}{TP + FN}, \quad \text{FPR} = \frac{FP}{FP + TN}
$$

- **AUC** mengukur seberapa baik model dapat memisahkan antara kelas positif dan kelas negatif. Nilai AUC mendekati 1 berarti model sangat baik dalam membedakan kedua kelas.
- **ROC Curve** adalah grafik yang menggambarkan hubungan antara TPR dan FPR pada berbagai threshold.

## 7. **Softmax Function**
Softmax digunakan pada model klasifikasi multi-kelas untuk menghasilkan probabilitas dari output yang tidak terdistribusi secara probabilistik.

$$
\text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}
$$

- \( z_i \) adalah skor mentah (logit) untuk kelas ke-\(i\).
- \( K \) adalah jumlah total kelas.
- Softmax mengubah nilai logit menjadi nilai probabilitas antara 0 dan 1, yang menjumlahkan seluruh probabilitas untuk kelas-kelas menjadi 1.

---

# Ringkasan
- **Cross-Entropy Loss** digunakan untuk menghitung seberapa baik model dalam memprediksi distribusi probabilitas untuk kelas-kelas.
- **Akurasi** menunjukkan seberapa sering model memprediksi dengan benar.
- **Precision** dan **Recall** mengukur kualitas prediksi positif, dengan Precision lebih fokus pada **keakuratan** prediksi positif dan Recall lebih pada **kemampuan mendeteksi** positif.
- **F1 Score** menggabungkan Precision dan Recall untuk memberikan satu metrik.
- **AUC** dan **ROC Curve** menggambarkan seberapa baik model membedakan antara kelas-kelas yang berbeda.

Metrik-metrik ini sangat penting dalam mengevaluasi performa model, terutama dalam klasifikasi yang melibatkan lebih dari dua kelas atau ketidakseimbangan antara kelas.
